Intake-STAC + sat-search
================

Commonly, we'll use an API to search a large STAC catalog to narrow down the data we want to work with. Here we'll demonstrate using the [sat-search](https://github.com/sat-utils/sat-search) library that uses https://www.element84.com/earth-search/.

In [ ]:
%matplotlib inline
import intake
import satsearch

In [ ]:
bbox = [35.48, -3.24, 35.58, -3.14] # (min lon, min lat, max lon, max lat)
dates = '2020-07-01/2020-08-15'

URL='https://earth-search.aws.element84.com/v0'
results = satsearch.Search.search(url=URL,
                                  collections=['sentinel-s2-l2a-cogs'], # note collection='sentinel-s2-l2a-cogs' doesn't work
                                  datetime=dates,
                                  bbox=bbox,    
                                  sort=['<datetime'])

In [ ]:
print('%s items' % results.found())
items = results.items()
# Save this locally for use later
items.save('my-s2-l2a-cogs.json')

In [ ]:
type(items)

In [ ]:
catalog = intake.open_stac_item_collection(items)

In [ ]:
list(catalog)

In [ ]:
item = catalog['S2A_36MYB_20200814_0_L2A']

In [ ]:
# Assets for STAC Item
list(item)

In [ ]:
from IPython.display import Image
Image(item['thumbnail'].urlpath)

In [ ]:
# full STAC JSON accessible as item._stac_obj._data
item.metadata

In [ ]:
da = item.B04(chunks=dict(band=1, y=2048, x=2048)).to_dask()
da

In [ ]:
# stack_bands() method should be identical to landsat
bands = ['nir','red']
stack = item.stack_bands(bands)
type(stack)

In [ ]:
# Bug? currently need to specify chunks:
da = stack(chunks=dict(band=1, x=2048, y=2048)).to_dask()
da

In [ ]:
# Reorganize into xarray DataSet with common band names
da['band'] = bands
ds = da.to_dataset(dim='band')
ds

In [ ]:
# Extract and plot NDVI subset
NDVI = (ds['nir'] - ds['red']) / (ds['nir'] + ds['red'])
#subset = NDVI.sel(y=slice(9.63e6,9.64e6), x=slice(7.55e5,7.71e5))
subset = NDVI.isel(y=slice(6000,7000), x=slice(4500,5500))
subset.plot.imshow(cmap='BrBG', vmin=-1, vmax=1)